In [1]:
import k3d.plot
import k3d.platonic
import typing
import json

In [2]:
class P(typing.NamedTuple):
    x: int
    y: int
    z: int

class Dog(typing.NamedTuple):
    points: list[P]
    color: int
        
class Position(typing.NamedTuple):
    freeSpace: list[P]
    dogs: list[Dog]

In [3]:
def drawDogs(dogs: list[Dog]) -> k3d.plot:
    plot = k3d.plot()
    for dog in dogs:
        for p in dog.points:
            mesh = k3d.platonic.Cube((p.x, p.y, p.z), 0.5).mesh
            mesh.color = dog.color
            plot += mesh
    return plot

In [4]:
def loadDogs(path: str) -> typing.Generator[Dog, None, None]:
    with open(path) as handle:
        data = json.loads(handle.read())
    
    for dogData in data:
        points = [P(a[0], a[1], a[2]) for a in dogData["points"]]
        color = int(dogData["color"], base = 16)
        yield Dog(points, color)
        
def loadPositions(path: str) -> typing.Generator[Position, None, None]:
    with open(path) as handle:
        data = json.loads(handle.read())
    
    for positionData in data:
        freeSpace = [P(a[0], a[1], a[2]) for a in positionData["free"]]
        yield Position(freeSpace, [])

In [5]:
# assuming one axis of symmetry (all the dog shapes are flat), so 12 rotations
def allDogRotations(dog: Dog) -> typing.Generator[Dog, None, None]:
    def rotateX(dog: Dog) -> Dog:
        newPoints = [P(p.x, p.z, -p.y) for p in dog.points]
        return Dog(newPoints, dog.color)
    
    def rotateY(dog: Dog) -> Dog:
        newPoints = [P(p.z, p.y, -p.x) for p in dog.points]
        return Dog(newPoints, dog.color)

    def rotateZ(dog: Dog) -> Dog:
        newPoints = [P(p.y, -p.x, p.z) for p in dog.points]        
        return Dog(newPoints, dog.color)
        
    for i in range(4):
        yield dog
        dog = rotateX(dog)
    
    dog = rotateY(dog)

    for i in range(4):
        yield dog
        dog = rotateZ(dog)

    dog = rotateX(dog)
    
    for i in range(4):
        yield dog        
        dog = rotateY(dog)

In [6]:
def allDogFits(position: Position, dog: Dog) -> typing.Generator[Position, None, None]:
    for rotatedDog in allDogRotations(dog):
        for fp in position.freeSpace:
            translatedDog = Dog([P(dp.x+fp.x, dp.y+fp.y, dp.z+fp.z) for dp in rotatedDog.points], dog.color)
            stickingOut = any([p for p in translatedDog.points if p not in position.freeSpace])
            
            if not stickingOut:
                newFreeSpace = [p for p in position.freeSpace if p not in translatedDog.points]
                yield Position(newFreeSpace, position.dogs + [translatedDog])

In [7]:
dogs = list(loadDogs("data/dogs.json"))
positions = list(loadPositions("data/positions.json"))

In [8]:
def solvePosition(position: Position, remainingDogs: list[Dog]) -> Position:
    if not remainingDogs:
        return position
    
    for newPosition in allDogFits(position, remainingDogs[0]):
        trySolutionPosition = solvePosition(newPosition, remainingDogs[1:])
        if trySolutionPosition:
            return trySolutionPosition

In [9]:
solution = solvePosition(positions[1], dogs)

In [10]:
drawDogs(solution.dogs)

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera=[2, -3, 0.2, 0.0, 0…